In [1]:
import sys
sys.path.append("..")

from glob import glob
from save_flight_frames import save_flight_frames
from utils.data_loading_utils import get_flight_files, get_era5_data

import datetime
import os
import time

In [2]:
# Settings
flight_id_filter = []   # If looking at specific flight(s)

dates = [(2023, 8, 5)]
hour_range = 10
num_frames_after_flight = 100

indiv_stations  = ['US001N']
all_stations = False

In [4]:
for (year, month, day) in dates:
    stations = []
    if all_stations:
        station_dirs = glob('../data/gmn/*/')
        for station_dir in station_dirs:
            station_name = os.path.basename(os.path.normpath(station_dir))
            if len(glob(f'{station_dir}/*{year}{month:02d}{day:02d}*')) == 1:
                stations.append(station_name)
                print(station_name)
            else:
                print(f"Skipping {station_name} for {year}-{month:02d}-{day:02d}")
    else:
        for station in indiv_stations:
            station_dir = f'../data/gmn/{station}'

            if len(glob(f'{station_dir}/*{year}{month:02d}{day:02d}*')) == 1:
                stations.append(station)
            else:
                print(f"Skipping {station} for {year}-{month:02d}-{day:02d} (could not find)")

    for station in stations:
        # try:
        output_dir = f'../data/gmn_extracted_flight_images/{station}/{year}{month:02d}{day:02d}'
        if os.path.exists(os.path.join(output_dir, 'done.txt')):
            print(f"Skipping {station} for {year}-{month:02d}-{day:02d} (already done)")
            continue
        
        video_dir = glob(f'../data/gmn/{station}/*{year}{month:02d}{day:02d}*/')[0]
        start_hour = int(video_dir.split('_')[2][:2])

        times = []
        for i in range(hour_range):
            start_time = datetime.datetime(year, month, day, start_hour, 0, 0)
            start_time += datetime.timedelta(hours=i)
            times.append((start_time.year, start_time.month, start_time.day, start_time.hour))
        
        # Get ERA5 data with some offset
        hour_offset = 3
        start_era5 = datetime.datetime(times[0][0], times[0][1], times[0][2], times[0][3], 0, 0)
        start_era5 -= datetime.timedelta(hours=hour_offset)

        end_era5 = datetime.datetime(times[-1][0], times[-1][1], times[-1][2], times[-1][3], 0, 0)
        end_era5 += datetime.timedelta(hours=hour_offset)

        start_year, start_month, start_day, start_hour = (start_era5.year, start_era5.month, start_era5.day, start_era5.hour)
        end_year, end_month, end_day, end_hour = (end_era5.year, end_era5.month, end_era5.day, end_era5.hour)
        file_path_met = f"../data/era5/{year}{month:02d}/Met*"

        met = get_era5_data(start_year, start_month, start_day, start_hour, end_year, end_month, end_day, end_hour, file_path_met)

        output_meta_file = f'../data/gmn_processed_flight_data/{year}{month:02d}{day:02d}_meta.pq'
        output_wypts_file = f'../data/gmn_processed_flight_data/{year}{month:02d}{day:02d}_wypts.pq'
        flight_data_files = get_flight_files(times, flight_data_base_path='../data/flight_data/')

        start_time = time.time()
        save_flight_frames(output_dir, 'gmn', video_dir, flight_data_files, output_meta_file, output_wypts_file, era5_met=met, num_frames_after_flight=num_frames_after_flight,
            flight_id_filter=flight_id_filter, cache_flight_data=False)
        
        print(f"{station} {year}-{month:02d}-{day:02d} took {time.time() - start_time} seconds")
        # except Exception as e:
            # print(f"Station {station} Error: {e}")

../data/gmn/US001N\US001N_20230805_003307_279816_detected\.config


NameError: name 'exit' is not defined